In [58]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
import pandas as pd
import plotly

df = pd.read_csv("202501_202501_주민등록인구및세대현황_월간.csv",encoding='cp949')
df

,행정구역,2025년01월_총인구수,2025년01월_세대수,2025년01월_세대당 인구,2025년01월_남자 인구수,2025년01월_여자 인구수,2025년01월_남여 비율
0,서울특별시 (1100000000),"9,330,658","4,482,949",2.08,"4,504,432","4,826,226",0.93
1,서울특별시 종로구 (1111000000),"138,208","72,156",1.92,"66,514","71,694",0.93
2,서울특별시 중구 (1114000000),"120,325","64,933",1.85,"57,948","62,377",0.93
3,서울특별시 용산구 (1117000000),"203,727","103,204",1.97,"97,392","106,335",0.92
4,서울특별시 성동구 (1120000000),"273,549","131,857",2.07,"132,354","141,195",0.94
...,...,...,...,...,...,...,...
287,경상남도 거창군 (4888000000),"59,514","31,098",1.91,"29,241","30,273",0.97
288,경상남도 합천군 (4889000000),"40,133","23,668",1.70,"19,738","20,395",0.97
289,제주특별자치도 (5000000000),"669,460","314,683",2.13,"334,794","334,666",1.00
290,제주특별자치도 제주시 (5011000000),"487,623","225,733",2.16,"243,336","244,287",1.00


In [60]:
# df의 "2025년01월_총인구수" 열의 데이터를 정수로 변환
df['2025년01월_총인구수'] = df['2025년01월_총인구수'].apply(lambda x: int(x.replace(',','')))
df = df[df['2025년01월_총인구수'] != 0]

# '행정구역'열 => '구역명','구역코드'로 분할
def split_code_name(x):
    splited = x.split('(')
    return pd.Series([splited[0].strip(), splited[1][0:5]])

df[['구역명','구역코드']] = df['행정구역'].apply(lambda x: split_code_name(x))
df['시도명'] = df['구역명'].apply(lambda x: x.split()[0])
# 시도명 축약(서울특별시 -> 서울, 경상남도 -> 경남)
def abbreviate_province(name):
    if name[-2:] in ['남도','북도']:
        return name[0] + name[2]
    else:
        return name[:2]
df['시도명(함축)'] = df['시도명'].apply(lambda x : abbreviate_province(x))

C:\Users\turing01\AppData\Local\Temp\ipykernel_16124\3647662725.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['구역명','구역코드']] = df['행정구역'].apply(lambda x: split_code_name(x))
C:\Users\turing01\AppData\Local\Temp\ipykernel_16124\3647662725.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['구역명','구역코드']] = df['행정구역'].apply(lambda x: split_code_name(x))
C:\Users\turing01\AppData\Local\Temp\ipykernel_16124\3647662725.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [ ]:
import json
geojson = json.load(open("법정구역_시군구.geojson", encoding='utf-8'))
# 구역코드: 구역명(한글)
sido_geojson = {feat['properties']['SIG_CD']: feat['properties']['SIG_KOR_NM'] for feat in geojson['features']}
df['구역명확인'] = df['구역코드'].map(sido_geojson)
# GeoJSON에 없는 구역들은 삭제해줍니다.
df = df[df['구역명확인'].notna()].copy().reset_index(drop=True)

# https://jgws.tistory.com/entry/Plotly%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%98%EC%97%AC-%EC%A7%80%EB%8F%84-%EC%8B%9C%EA%B0%81%ED%99%94-%ED%95%98%EA%B8%B0-Choropleth-Map-%EA%B7%B8%EB%A6%AC%EA%B8%B0
# TODO: 링크에 있는 사이트에 들어가서 plotly로 지도 그래프 완성하기

C:\Users\turing01\AppData\Local\Temp\ipykernel_16124\167378217.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['구역명확인'] = df['구역코드'].map(sido_geojson)


In [63]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Choropleth(geojson=geojson,
                                    featureidkey='properties.SIG_CD',#GeoJSON에서 활용할 id키
                                    locationmode= 'geojson-id',
                                    locations=df['구역코드'],
                                    z = df['2025년01월_총인구수'],
                                    colorscale = [[0, '#F1EFEF'], [0.2, '#CCC8AA'],   [0.4, '#B0AA7C'],  [0.6, '#7D7C7C'], [0.8,'#606060'], [1, '#191717']],
                                    # 컬러스케일은 원하는 갯수의 색상으로 구성할 수 있습니다.
                                    hovertext = df['구역명확인']+": " +df['2025년01월_총인구수'].apply(lambda x: f'{x:,} 명'),
                                    hoverinfo = 'text',
                                    marker_line_color='#555',
                                    # marker_line : 행정구역의 경계선
                                    marker_line_width=1,                            
                                    )
                )
                
fig.update_layout(geo = dict(fitbounds="locations", # 설정하지 않으면 세계지도가 표시됩니다
                             visible = False, # 설정하지 않으면 세계지도가 표시됩니다
                             showframe=False,
                             ),
                  margin={"r":0,"t":0,"l":0,"b":0},
                  )
                  
html = fig.to_html()
with open('시군구별 인구 현황_202308.html', "w", encoding="utf-8") as f:
    f.write(html)
# html 파일로 export 해줍니다.